In [1]:
!pip3 install beautifulsoup4
!pip install selenium
!pip3 install keybert
!pip install newspaper3k

In [6]:
import requests 
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time
import numpy
import random
import pickle
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
from newspaper import Article
import os
from keybert import KeyBERT

In [2]:
driver = webdriver.Chrome (executable_path=r"C:\Users\Notebiz003\Documents\chromedriver_win32\chromedriver.exe")

C:\Users\NH\AppData\Local\Temp\ipykernel_25472\3290017851.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome (executable_path=r"C:\Users\Notebiz003\Documents\chromedriver_win32\chromedriver.exe")


In [7]:
from_date = '2023-1-30'
to_date = '2023-2-5'
searching_word = 'metaverse'

cwd = os.getcwd()
main_path = cwd + '/' + searching_word

In [4]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        


createFolder(main_path)
createFolder(main_path + '/' + 'url')
createFolder(main_path + '/' + 'news_backup')
createFolder(main_path + '/' + 'news')
createFolder(main_path + '/' + 'keyword')

# 1 키워드 검색 뉴스 url 수집
def page2url(searching_word,date):
    urls = []
    for start in range(0, 360, 10):
        main_url = 'https://www.google.co.kr/search?q={}&tbs=cdr:1,cd_min:{},cd_max:{}&tbm=nws&ei=dPP-Yu_eCJLL-Qb55bvQDA&start={}&sa=N&ved=2ahUKEwjv6Lvy69H5AhWSZd4KHfnyDso4ChDy0wN6BAgBEDk&biw=1536&bih=754&dpr=1.25'.format(searching_word,date,date,start)
        driver.get(url=main_url)
        elements = driver.find_elements(By.TAG_NAME, 'a')
        lnks = []
        for lnk in elements:
            lnk = str(lnk.get_attribute('href'))
            if 'google' not in lnk and lnk != 'None':
                lnks.append(lnk)
        if len(lnks) == 0:
            print(date, ' Url 수집 완료')
            break
        urls.extend(lnks)
        rand_value =random.uniform(2, 4)
        time.sleep(rand_value)
        
    return urls

datelist = pd.date_range(start=from_date, end=to_date).tolist()
dtlst = []
for d_t in datelist:
    d_t = str(d_t)[0:-9]
    d = datetime.strptime(d_t, '%Y-%m-%d')
    d = d.strftime('%m/%d/%Y')
    d = d[0].replace('0','') + d[1:]
    d = d[:-7] + d[-7].replace('0','') + d[-6:]
    dtlst.append(d)
    
    
Urls = dict()
for date in tqdm(dtlst):
    urls = page2url(searching_word,date)
    Urls[date] = urls
    
name = "Urls{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'url/' + name,'wb') as f:
    pickle.dump(Urls,f)

# 2  수집된 url로 news 크롤링 
with open(main_path + '/' + 'url/' + name, 'rb') as f:
    data = pickle.load(f)

# start = 0
for date in list(data):
    cnt = 0
    tmp = []
    for idx,url in tqdm(enumerate(data[date])):
        try:
            article = Article(url)
            article.download()
            article.parse()
            v = article.text
            tmp.append(v)
            cnt += 1
        except:
            pass
            # print(url)
    data[date] = [tmp]
    print("{}에 총 {}건 중 {} 건 크롤링 완료".format(date,idx+1,cnt))
    date = date.replace('/','-')
    with open(main_path + '/' + 'news_backup/' + 'news-' + date + '.pickle','wb') as f:
        pickle.dump(data,f)
    
data = pd.DataFrame(data).T

data = data.rename({0:"news"},axis= 1)

name = "News{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'news/' + name,'wb') as f:
    pickle.dump(data,f)
    
# 3 news에서 키워드를 추출 (keybert)

def doc2key(cleaned_content):
    
    kw_model = KeyBERT(model='all-MiniLM-L6-v2')
    keywords = kw_model.extract_keywords(doc)
    
    n2_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(2, 2), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=5)

    n1_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(1, 1), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=10)
    for idx,i in enumerate(n2_kwd):
        n2_kwd[idx] = i[0]
    for idx,i in enumerate(n1_kwd):
        n1_kwd[idx] = i[0]  

    n1_kwd.extend(n2_kwd)
    kwd = n1_kwd
    return kwd

with open(main_path + '/' + 'news/' + name, 'rb') as f:
    data = pickle.load(f)
    
data = data.reset_index()
data = data.rename({"index":"date"},axis = 1)

idx = []
for x in data['date']:
    tmp = x.split('/')
    if len(tmp[0]) == 1:
        tmp[0] = '0'+tmp[0]
    if len(tmp[1]) == 1:
        tmp[1] = '0'+tmp[1]
    tmp = tmp[2] + tmp[0] + tmp[1]
    idx.append(''.join(tmp))
    
data['date'] = idx
data = data.rename({'date':'일자'},axis = 1)

kwd_list = []
for docs in tqdm(data['news']):
    kwds = []
    for doc in docs:
        try:
            cleaned_content = re.sub(r'[^\.\?\!\w\d\s]','',doc) # 문장단위로 끊기
            cleaned_content = cleaned_content.replace('\n',' ')
            cleaned_content = cleaned_content.lower()
            kwd = doc2key(cleaned_content)
            kwds.append(kwd)
        except:
            print(doc)
    kwd_list.append(kwds)
    print("Keyword 추출 중...")
print("Keyword 추출 완료")
    
data['kwd'] = kwd_list

name = "Keyword{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'keyword/' + name,'wb') as f:
    pickle.dump(data,f)


 14%|████████████                                                                        | 1/7 [01:35<09:34, 95.79s/it]

1/30/2023  Url 수집 완료


 29%|████████████████████████                                                            | 2/7 [03:06<07:43, 92.79s/it]

1/31/2023  Url 수집 완료


 43%|████████████████████████████████████                                                | 3/7 [04:26<05:48, 87.02s/it]

...

Keyword 추출 중...
Keyword 추출 완료
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출

 71%|███████████████████████████████████████████████████████████▎                       | 5/7 [28:58<11:18, 339.36s/it]

Keyword 추출 중...
Keyword 추출 완료
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출 중...
Keyword 추출

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [34:54<00:00, 299.26s/it]

Keyword 추출 중...
Keyword 추출 완료
Keyword 추출 완료


In [31]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\NH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\NH\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NH\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
from collections import Counter
import itertools

In [56]:
name = "Keyword{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'keyword/' + name, 'rb') as f:
    data = pickle.load(f)

In [57]:
df = data[['일자','kwd']]

In [58]:
# 키워드 단수 복수 통일
from nltk.stem import WordNetLemmatizer
lemmatizer = nltk.WordNetLemmatizer()

In [59]:
lemmatied_list = []
for i in df['kwd']:
    tmp = [lemmatizer.lemmatize(word) for word in itertools.chain.from_iterable(i)]    
    lemmatied_list.append(tmp)

In [60]:
df['kwd'] = lemmatied_list

C:\Users\NH\AppData\Local\Temp\ipykernel_16848\2507666250.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kwd'] = lemmatied_list


In [61]:
# 일별 빈도 카운트
keyword_vocab = []
for i in df['kwd']:
    keyword_vocab.extend(i)
keyword_vocab = list(set(keyword_vocab))
len(keyword_vocab)

11314

In [62]:
df_count = pd.DataFrame(index = keyword_vocab)

In [63]:
for idx in range(len(df)):
    tmp = pd.DataFrame({df['일자'][idx] : Counter(df['kwd'][idx])})
    df_count = df_count.join(tmp)

In [64]:
df_count = df_count.fillna(0)

In [65]:
df_count['tot'] = df_count.sum(axis = 1).values

In [66]:
df_count.sort_values(by = 'tot', ascending = False)[0:50]

,20230130,20230131,20230201,20230202,20230203,20230204,20230205,tot
metaverse,46.0,62.0,53.0,54.0,47.0,45.0,0.0,307.0
2023,42.0,46.0,35.0,32.0,38.0,24.0,0.0,217.0
2022,17.0,13.0,11.0,17.0,14.0,12.0,0.0,84.0
innovation,4.0,10.0,7.0,9.0,8.0,7.0,0.0,45.0
ecosystem,9.0,12.0,4.0,5.0,7.0,7.0,0.0,44.0
startup,8.0,10.0,6.0,3.0,2.0,8.0,0.0,37.0
billion,6.0,4.0,7.0,5.0,6.0,8.0,0.0,36.0
especially,10.0,6.0,4.0,3.0,7.0,2.0,0.0,32.0
digital,1.0,6.0,8.0,5.0,8.0,4.0,0.0,32.0
cryptocurrencies,4.0,4.0,6.0,5.0,5.0,5.0,0.0,29.0


In [63]:
# 4. 워드클라우드

In [76]:
df_count.sum(axis = 1)

market                 1.0
superheroeswhatever    1.0
plato digital          1.0
2023 roadmap           1.0
control bigg           1.0
                      ... 
chairman               2.0
maps designed          1.0
doha learning          1.0
com4k3dgzol12          1.0
im april               1.0
Length: 11532, dtype: float64

In [86]:
df_count.sort_values(by = 'tot', ascending = False)[0:50]

,20230130,20230131,20230201,20230202,20230203,20230204,20230205,tot
metaverse,46.0,62.0,53.0,54.0,47.0,45.0,0.0,307.0
2023,42.0,46.0,35.0,32.0,38.0,24.0,0.0,217.0
2022,17.0,13.0,11.0,17.0,14.0,12.0,0.0,84.0
ecosystem,7.0,11.0,4.0,5.0,7.0,7.0,0.0,41.0
especially,10.0,6.0,4.0,3.0,7.0,2.0,0.0,32.0
digital,1.0,6.0,8.0,5.0,8.0,4.0,0.0,32.0
zuckerberg,2.0,2.0,6.0,8.0,5.0,6.0,0.0,29.0
nfts,5.0,7.0,5.0,5.0,5.0,2.0,0.0,29.0
cryptocurrencies,4.0,4.0,6.0,5.0,5.0,5.0,0.0,29.0
vr,5.0,8.0,5.0,4.0,3.0,3.0,0.0,28.0


In [ ]:
# 5. 연관도 분석

## 날짜와 상관없이 가중치 매트릭스